# Поиск аномалий

Методы обнаружения аномалий, как следует из названия, позволяют находить необычные объекты в выборке. Но что такое "необычные" и совпадает ли это определение у разных методов?

Начнём с поиска аномалий в текстах: научимся отличать вопросы о программировании от текстов из 20newsgroups про религию.

Подготовьте данные: в обучающую выборку возьмите 20 тысяч текстов из датасета Stack Overflow, а тестовую выборку сформируйте из 10 тысяч текстов со Stack Overflow и 100 текстов из класса soc.religion.christian датасета 20newsgroups (очень пригодится функция `fetch_20newsgroups(categories=['soc.religion.christian'])`). Тексты про программирование будем считать обычными, а тексты про религию — аномальными.

In [17]:
import numpy as np
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
s_texts = pd.read_csv("StackOverflow_posts.csv")
s_texts

,Body
0,<p>I'm new to C# and I want to use a track-bar...
1,<p>I have an absolutely positioned <code>div</...
2,<p>An explicit cast to double isn't necessary....
3,<p>Are there any conversion tools for porting ...
4,<p>Given a <code>DateTime</code> representing ...
...,...
29995,<p>But it's FUN! I mean it's more interesting ...
29996,<p>It depends on the number of images you are ...
29997,<p>I'm trying to find a simple way to change t...
29998,"<p><strong><em><a href=""http://dotnetnuke.mont..."


In [18]:
s_texts['flag'] = [1 for i in range(30000)]
s_texts

,Body,flag
0,<p>I'm new to C# and I want to use a track-bar...,1
1,<p>I have an absolutely positioned <code>div</...,1
2,<p>An explicit cast to double isn't necessary....,1
3,<p>Are there any conversion tools for porting ...,1
4,<p>Given a <code>DateTime</code> representing ...,1
...,...,...
29995,<p>But it's FUN! I mean it's more interesting ...,1
29996,<p>It depends on the number of images you are ...,1
29997,<p>I'm trying to find a simple way to change t...,1
29998,"<p><strong><em><a href=""http://dotnetnuke.mont...",1


In [19]:
newsgroups = fetch_20newsgroups(categories=['soc.religion.christian'], remove=("headers", "footers", "quotes"))
r_texts = newsgroups.data[:100]
r_texts = pd.DataFrame.from_dict({"Body": r_texts, "flag":[-1 for i in range(len(r_texts))]})
r_texts

,Body,flag
0,I wrote in response to dlecoint@garnet.acns.fs...,-1
1,"A ""new Christian"" wrote that he was new to the...",-1
2,: > \t I'm a commited Christian that is batt...,-1
3,My brother has been alienated from my parents ...,-1
4,"> [A very nice article on the DSS, which I ...",-1
...,...,...
95,I'd like to share my thoughts on this topic of...,-1
96,Sorry for bothering with a request almost irre...,-1
97,\nmy $.02 - Yes and No. I do not believe the ...,-1
98,I just about closed this once before. I'm now...,-1


In [20]:
s_texts = pd.concat([s_texts, r_texts], ignore_index=True)

In [21]:
s_texts

,Body,flag
0,<p>I'm new to C# and I want to use a track-bar...,1
1,<p>I have an absolutely positioned <code>div</...,1
2,<p>An explicit cast to double isn't necessary....,1
3,<p>Are there any conversion tools for porting ...,1
4,<p>Given a <code>DateTime</code> representing ...,1
...,...,...
30095,I'd like to share my thoughts on this topic of...,-1
30096,Sorry for bothering with a request almost irre...,-1
30097,\nmy $.02 - Yes and No. I do not believe the ...,-1
30098,I just about closed this once before. I'm now...,-1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(s_texts.drop('flag', axis=1), s_texts.flag, test_size=10100, random_state=42, shuffle=False)

In [23]:
y_test

20000    1
20001    1
20002    1
20003    1
20004    1
        ..
30095   -1
30096   -1
30097   -1
30098   -1
30099   -1
Name: flag, Length: 10100, dtype: int64

In [24]:
X_train = X_train.Body.to_list()
y_train = y_train.to_list()
X_test = X_test.Body.to_list()
y_test = y_test.to_list()

**(1 балл)**

Проверьте качество выделения аномалий (precision и recall на тестовой выборке, если считать аномалии положительным классов, а обычные тексты — отрицательным) для IsolationForest. В качестве признаков используйте TF-IDF, где словарь и IDF строятся по обучающей выборке. Не забудьте подобрать гиперпараметры.

In [25]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [26]:
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection

isolation_forest = IsolationForest(random_state=42)
param_grid = {'n_estimators': [50, 100, 150],
              'max_samples': ['auto', 0.1, 0.6, 0.8], 
              'contamination': [0.005], 
              'max_features': [10, 100, 1000, 5000, 10000], 
              'n_jobs' : [-1],
              'bootstrap': [True, False]}

#f1sc = make_scorer(f1_score(average='micro'))
grid_dt_estimator = model_selection.GridSearchCV(isolation_forest, 
                                                 param_grid,
                                                 scoring='f1', 
                                                 refit=True,
                                                 cv=4, 
                                                 return_train_score=True, verbose=100)
#grid_dt_estimator.fit(X_train, y_train)
isolation_forest.fit(X_train)

test_predictions = isolation_forest.predict(X_test)

precision = precision_score(y_test, test_predictions, average='micro')
recall = recall_score(y_test, test_predictions, average='micro')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Precision: 0.9901
Recall: 0.9901


**(5 баллов)**

Скорее всего, качество оказалось не на высоте. Разберитесь, в чём дело:
* посмотрите на тексты, которые выделяются как аномальные, а также на слова, соответствующие их ненулевым признакам
* изучите признаки аномальных текстов
* посмотрите на тексты из обучающей выборки, ближайшие к аномальным; действительно ли они похожи по признакам?

Сделайте выводы и придумайте, как избавиться от этих проблем. Предложите варианты двух типов: (1) в рамках этих же признаков (но которые, возможно, будут считаться по другим наборам данных) и методов и (2) без ограничений на изменения. Реализуйте эти варианты и проверьте их качество.

In [27]:
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
feature_names

array(['00', '000', '0000', ..., 'zoo', 'zoom', 'zope'], dtype=object)

In [28]:
import numpy as np

anomalies = [i for i, pred in enumerate(test_predictions) if pred == -1]
anomalous_texts = [X_test[i] for i in anomalies]
anomalous_indices = anomalies


anomalous_features = {}
top_by_freq = {}
top_by_ws = {}
for idx in anomalous_indices:
    feature_vector = X_test[idx].toarray().flatten()
    
    non_zero_indices = np.where(feature_vector > 0)[0]
    non_zero_words = [(feature_names[i], feature_vector[i]) for i in non_zero_indices]
    anomalous_features[idx] = non_zero_words
    for word, score in anomalous_features[idx]:
        if word not in top_by_freq:
            top_by_freq[word] = 1
            top_by_ws[word] = score
        else:
            top_by_freq[word] += 1
            top_by_ws[word] = max(score, top_by_ws[word])

top_by_freq = list(top_by_freq.items())
top_by_ws = list(top_by_ws.items())
top_by_freq.sort(key=lambda x : -x[1])
top_by_freq_words = list(map(lambda x : x[0], top_by_freq))
top_by_freq_ks = list(map(lambda x : x[1], top_by_freq))
top_by_ws.sort(key=lambda x : -x[1])
top_by_ws_words = list(map(lambda x : x[0], top_by_ws))
top_by_ws_ks = list(map(lambda x : x[1], top_by_ws))

In [30]:
np.where(test_predictions==-1)

(array([], dtype=int64),)

In [31]:
top_by_freq = pd.DataFrame.from_dict({'word' : top_by_freq_words, 'score' : top_by_freq_ks})
top_by_freq

,word,score


In [32]:
top_by_ws = pd.DataFrame.from_dict({'word' : top_by_ws_words, 'score' : top_by_ws_ks})
top_by_ws

,word,score
